In [1]:
from probe_experiment import *
from handcrafted_datasets import Dataset
from models import *
import torch
import torch.nn as nn
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
dataset = Dataset('last_move', 'C:\\Users\\andre\\go-ai\\data', 0.8, 0.2, 1024)

def get_train_loader():
    dataset.shuffle('train')
    return dataset.loader('train', max_ram_files=100)

def get_test_loader():
    return dataset.loader('test', max_ram_files=100)

done loading data
split sizes:
train 552
test 138


In [3]:
import numpy as np
mht_dist_grid = np.zeros((39, 39))
for dist in range(39):
    for i in range(dist+1):
        j = dist - i
        for r1, c1 in ((i, j), (i, -j), (-i, j), (-i, -j)):
            r, c = r1+19, c1+19
            if 0 <= r < 39 and 0 <= c < 39:
                mht_dist_grid[r][c] = dist

def mht_grid_19x19(center_r, center_c):
    return mht_dist_grid[19-center_r:38-center_r, 19-center_c:38-center_c]     

mht_grids = {pt:torch.Tensor(mht_grid_19x19(pt//19, pt%19)).detach() for pt in range(19*19)}
                
def weighted_mht_dist_loss(pred, target):
    mht = [mht_grids[int(pt.data.item())] for pt in target]
    mht = torch.stack(mht)                  
    mht_flat = nn.Flatten()(mht).cuda()
    pred = nn.Softmax()(nn.Flatten()(pred))
    return torch.mean(torch.sum(pred * mht_flat, dim=1))

criterion = weighted_mht_dist_loss

In [4]:
'''
r, c = 5, 18
p = np.ones((1, 19, 19)) * -100
p[0][r+2][c-1] = 100
weighted_mht_dist_loss(torch.from_numpy(p), r*19+c)
'''

'\nr, c = 5, 18\np = np.ones((1, 19, 19)) * -100\np[0][r+2][c-1] = 100\nweighted_mht_dist_loss(torch.from_numpy(p), r*19+c)\n'

In [5]:
go_model = load_go_model_from_ckpt('model_ckpt.pth.tar', rm_prefix=True)
feat_models = [CutModel(go_model, i).cuda() for i in range(8)]
n_channels = [8, 64, 64, 64, 48, 48, 32, 32]
probe_models = [nn.Sequential(nn.Conv2d(nc, 1, 39, padding=19), nn.Flatten()).cuda() for nc in n_channels]

In [ ]:
losses = []
for depth in range(3, 4):
    print('probing at depth %d' % depth)
    feat_model = feat_models[depth]
    probe_model = probe_models[depth]
    exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
    config = {'name':'lmv/lmv_19x19cnn_d%d'%depth, 'write_log':True, 'progress_bar':True, 'save_ckpt':False}
    optimizer = torch.optim.Adam(probe_model.parameters())
    exp.run(probe_model, criterion, optimizer, 10, config)
    

probing at depth 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 552/552 [16:32<00:00,  1.80s/it]


[LOG] epoch 0 loss 6.049851, new best


  0%|                                                                                                                            | 0/552 [00:00<?, ?it/s]

[LOG] epoch 1 loss 5.813234, new best


 15%|█████████████████▌                                                                                                 | 84/552 [01:49<09:41,  1.24s/it]

### feature: change of stone